In [4]:
import pyspark
from pyspark.ml.feature import StringIndexer, OneHotEncoder,VectorAssembler, Normalizer
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

In [46]:
df=spark.read.parquet('hmp.parquet')
df

DataFrame[x: int, y: int, z: int, source: string, class: string]

In [14]:
df = df2.withColumnRenamed("_c0","x")
df

DataFrame[x: string, _c1: string, _c2: string, _c3: string, _c4: string]

In [15]:
df = df2.withColumnRenamed("_c1","y")
df

DataFrame[_c0: string, y: string, _c2: string, _c3: string, _c4: string]

In [17]:
df3=df.withColumnRenamed("_c0", "x")
df3

DataFrame[x: string, y: string, _c2: string, _c3: string, _c4: string]

In [19]:
df4=df3.withColumnRenamed("_c2", "z")
df4

DataFrame[x: string, y: string, z: string, _c3: string, _c4: string]

In [20]:
df5=df4.withColumnRenamed("_c3", "source")
df5

DataFrame[x: string, y: string, z: string, source: string, _c4: string]

In [21]:
df6=df5.withColumnRenamed("_c4", "class")
df6

DataFrame[x: string, y: string, z: string, source: string, class: string]

In [22]:
df=df6
df

DataFrame[x: string, y: string, z: string, source: string, class: string]

In [47]:
#lets create pipeline
indexer=StringIndexer(inputCol="class",outputCol="label")
vectorAssembler=VectorAssembler(inputCols=["x","x","x"], outputCol="features")
normalizer=Normalizer(inputCol="features", outputCol="features_norm")

In [48]:
from pyspark.ml.classification import LogisticRegression
lr=LogisticRegression(maxIter=10,regParam=0.3, elasticNetParam=0.8)

In [49]:
from pyspark.ml import Pipeline
pipeline=Pipeline(stages=[indexer,vectorAssembler,normalizer,lr])

In [61]:
#lets split data
splits=df.randomSplit([0.8,0.2])
df_train=splits[0]
df_test=splits[1]

​

SyntaxError: invalid character in identifier (<ipython-input-61-b4cc29942adc>, line 9)

In [51]:
#now create model
model=pipeline.fit(df_train)
prediction=model.transform(df_train)

In [53]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eval=MulticlassClassificationEvaluator().setMetricName('accuracy').setLabelCol('label').setPredictionCol('prediction')
eval.evaluate(prediction)

0.20677230204556646